In [1]:
#!pip install scrapy

In [6]:
import pandas as pd
import numpy as np
import scrapy
import os
import logging
from scrapy.crawler import CrawlerProcess
import time
import json

In [3]:
weather_df = pd.read_csv('weather_data.csv')
weather_df

,id,city,feels_like.day,pop,delta_with_ideal_temp,pop_%,delta_temp_scaled,pop_scaled,score_to_minimize,rank
0,0,Eguisheim,30.43625,0.12375,0,12.375,-1.288998,-0.568045,-1.857044,1
1,1,Besancon,30.28750,0.17500,0,17.500,-1.288998,-0.291162,-1.580160,2
2,2,Strasbourg,30.63875,0.08375,1,8.375,-0.793230,-0.784149,-1.577379,3
3,3,Dijon,31.17000,0.08875,1,8.875,-0.793230,-0.757136,-1.550366,4
4,4,Cassis,30.25500,0.19250,0,19.250,-1.288998,-0.196616,-1.485615,5
5,5,Marseille,30.36500,0.19375,0,19.375,-1.288998,-0.189863,-1.478861,6
6,6,Rouen,28.34375,0.01125,2,1.125,-0.297461,-1.175838,-1.473299,7
7,7,Colmar,30.60375,0.12000,1,12.000,-0.793230,-0.588305,-1.381535,8
8,8,Annecy,30.20000,0.22375,0,22.375,-1.288998,-0.027785,-1.316783,9
9,9,Aigues Mortes,30.45250,0.24375,0,24.375,-1.288998,0.080267,-1.208731,10


In [4]:
top_cities = weather_df['city'].to_list()[:5]
top_cities

['Eguisheim', 'Besancon', 'Strasbourg', 'Dijon', 'Cassis']

In [5]:
HOTELS_NB = 20

class HotelsInfoSpider(scrapy.Spider):
    # Name of your spider
    name = "Hotels_Scraping"

    # Starting URL
    start_urls = [f'https://www.booking.com/searchresults.fr.html?ss={city}' for city in top_cities]

    def start_requests(self): # overriding the default start_requests method to keep track of the city as metadata
        for url, city in zip(self.start_urls, top_cities):
            yield scrapy.Request(url, meta={'city': city})
    
    # Parse method for searching most recommended hotels in each city
    def parse(self, response):
        for link in response.css('h3 a[data-testid="title-link"]')[:HOTELS_NB]:
            hotel_url = link.attrib["href"]
            yield response.follow(hotel_url, callback=self.parse_hotel, meta={'city': response.meta['city']}) # keeping track of the city metadata for each hotel
    
    # Parse method for collecting informations on each hotel
    def parse_hotel(self, response):
        yield {
            'city' : response.meta['city'],
            'hotel_name' : ''.join(response.css('div.hp__hotel-title h2::text').getall()), # some texts contain linebreaks interpreted as separators 
            'booking_url' : response.url,
            'lat_lon coordinates' : response.css('a#hotel_sidebar_static_map').attrib['data-atlas-latlng'],
            'booking_reviews_score' : response.css('div[data-testid="review-score-component"] div:first-child::text').get(), 
            'description' : response.css('#property_description_content p::text').getall() # getting all paragraphs except the two first ones 
                                                                                                          # (which are incentives to login booking.com writes to users)
        }

# Name of the file where the results will be saved
filename = "top_hotels.json"

# If file already exists, delete it before crawling (because Scrapy will 
# concatenate the last and new results otherwise)
if filename in os.listdir('src/'):
        os.remove('src/' + filename)

# Declare a new CrawlerProcess with some settings
## USER_AGENT => Simulates a browser on an OS
## LOG_LEVEL => Minimal Level of Log 
## FEEDS => Where the file will be stored 
## More info on built-in settings => https://docs.scrapy.org/en/latest/topics/settings.html?highlight=settings#settings
process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'src/' + filename: {"format": "json"},
    }
    #"FEED_EXPORT_ENCODING" : 'utf-8'
    #"AUTOTHROTTLE_ENABLED": True  # AutoThrottle Here!
    #"ROBOTSTXT_OBEY" : False, 
    #"COOKIES_ENABLED" : True,
    #"CONCURRENT_REQUESTS" : 1,
    #"DOWNLOAD_DELAY" : 1
    #"BOT_NAME" : "eve"
})

# Start the crawling using the spider you defined above
process.crawl(HotelsInfoSpider)
process.start()

2022-08-03 19:01:50 [scrapy.utils.log] INFO: Scrapy 2.6.2 started (bot: scrapybot)
2022-08-03 19:01:50 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:20:46) - [GCC 9.4.0], pyOpenSSL 22.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 36.0.1, Platform Linux-5.4.188+-x86_64-with-glibc2.31
2022-08-03 19:01:50 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
               '(KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
2022-08-03 19:01:50 [scrapy.extensions.telnet] INFO: Telnet Password: f0fd2ff2b58a794b
2022-08-03 19:01:50 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.e

In [33]:
with open("src/top_hotels.json", 'r') as top_hotels_file:
    top_hotels_df = pd.DataFrame(json.load(top_hotels_file))
top_hotels_df

,city,hotel_name,booking_url,lat_lon coordinates,booking_reviews_score,description
0,Besancon,\nibis Besançon Centre Ville\n,https://www.booking.com/hotel/fr/ibis-besancon...,"47.24045605,6.02599533","8,4",[L'ibis Besançon Centre Ville se trouve dans l...
1,Besancon,\nContact Hôtel Foch\n,https://www.booking.com/hotel/fr/foch-besancon...,"47.24619696,6.02335095","7,0",[Situé à quelques pas de la gare de Besançon e...
2,Besancon,\nPremiere Classe Besancon Ecole Valentin\n,https://www.booking.com/hotel/fr/premiere-clas...,"47.27044104,5.99827230","6,8","[Situé à proximité de l'Alsace et du Jura, cet..."
3,Besancon,\nAPPARTEMENT CENTRE HISTORIQUE\n,https://www.booking.com/hotel/fr/appartement-c...,"47.23432450,6.02773470","8,5",[L'APPARTEMENT CENTRE HISTORIQUE est situé dan...
4,Besancon,\nHotel Vauban\n,https://www.booking.com/hotel/fr/vauban-besanc...,"47.23986296,6.02139884","8,5","[L'Hotel Vauban vous accueille à Besançon, à 2..."
...,...,...,...,...,...,...
95,Besancon,\nHôtel de Paris\n,https://www.booking.com/hotel/fr/de-paris-besa...,"47.23856537,6.02532506","8,5","[Situé dans le centre de Besançon, l'Hôtel de ..."
96,Besancon,\nMercure Besancon Parc Micaud\n,https://www.booking.com/hotel/fr/besancon-parc...,"47.24228752,6.03179455","7,7",[Cet hôtel Mercure est situé dans le centre-vi...
97,Besancon,\nBest Western Citadelle\n,https://www.booking.com/hotel/fr/bestwesternci...,"47.23154100,6.02658033","8,2",[Le Best Western Citadelle est situé dans le c...
98,Besancon,\nHôtel AKENA BESANCON\n,https://www.booking.com/hotel/fr/akena-besanco...,"47.23547525,5.90008540","8,6",[Vous pouvez bénéficier d'une réduction Genius...


In [34]:
top_hotels_df.dtypes

city                     object
hotel_name               object
booking_url              object
lat_lon coordinates      object
booking_reviews_score    object
description              object
dtype: object

In [35]:
top_hotels_df['hotel_name'] = top_hotels_df['hotel_name'].str.strip()
top_hotels_df['latitude'] = top_hotels_df['lat_lon coordinates'].str.split(',').apply(lambda x: x[0]).astype(float)
top_hotels_df['longitude'] = top_hotels_df['lat_lon coordinates'].str.split(',').apply(lambda x: x[1]).astype(float)
top_hotels_df['booking_reviews_score'] = top_hotels_df['booking_reviews_score'].str.replace(',','.').astype(float)
top_hotels_df['description'] = top_hotels_df['description'].str.join('\n')
top_hotels_df = top_hotels_df.drop(columns = ['lat_lon coordinates'])
top_hotels_df

,city,hotel_name,booking_url,booking_reviews_score,description,latitude,longitude
0,Besancon,ibis Besançon Centre Ville,https://www.booking.com/hotel/fr/ibis-besancon...,8.4,L'ibis Besançon Centre Ville se trouve dans la...,47.240456,6.025995
1,Besancon,Contact Hôtel Foch,https://www.booking.com/hotel/fr/foch-besancon...,7.0,Situé à quelques pas de la gare de Besançon et...,47.246197,6.023351
2,Besancon,Premiere Classe Besancon Ecole Valentin,https://www.booking.com/hotel/fr/premiere-clas...,6.8,"Situé à proximité de l'Alsace et du Jura, cet ...",47.270441,5.998272
3,Besancon,APPARTEMENT CENTRE HISTORIQUE,https://www.booking.com/hotel/fr/appartement-c...,8.5,L'APPARTEMENT CENTRE HISTORIQUE est situé dans...,47.234324,6.027735
4,Besancon,Hotel Vauban,https://www.booking.com/hotel/fr/vauban-besanc...,8.5,"L'Hotel Vauban vous accueille à Besançon, à 2 ...",47.239863,6.021399
...,...,...,...,...,...,...,...
95,Besancon,Hôtel de Paris,https://www.booking.com/hotel/fr/de-paris-besa...,8.5,"Situé dans le centre de Besançon, l'Hôtel de P...",47.238565,6.025325
96,Besancon,Mercure Besancon Parc Micaud,https://www.booking.com/hotel/fr/besancon-parc...,7.7,Cet hôtel Mercure est situé dans le centre-vil...,47.242288,6.031795
97,Besancon,Best Western Citadelle,https://www.booking.com/hotel/fr/bestwesternci...,8.2,Le Best Western Citadelle est situé dans le ce...,47.231541,6.026580
98,Besancon,Hôtel AKENA BESANCON,https://www.booking.com/hotel/fr/akena-besanco...,8.6,Vous pouvez bénéficier d'une réduction Genius ...,47.235475,5.900085


In [40]:
print("------------------------------ Showing description of randomly picked hotels : --------------------------------------\n")
[print(descr, "\n----------------------------------------------------------------\n") for descr in top_hotels_df['description'].sample(10)]
print()

------------------------------ Showing description of randomly picked hotels : --------------------------------------

Situé dans le centre-ville de Dijon, à seulement 3 minutes à pied de la place de la République, l'Aparthotel Adagio Access Dijon République propose une réception ouverte 24h/24 et une connexion Wi-Fi gratuite dans l'ensemble de ses locaux. Le musée des Beaux-Arts est à 15 minutes à pied.
Climatisées, tous les hébergements disposent d'une pièce à vivre avec une télévision, d'un bureau et d'une salle de bains munie d'un sèche-cheveux. Leur cuisine entièrement équipée possède un micro-ondes, des plaques de cuisson et un réfrigérateur.
Vous pourrez préparer des repas faits maison dans le studio ou l'appartement. Un petit-déjeuner buffet est servi chaque matin dans la salle prévue à cet effet de l'établissement, tandis que les enfants de 4 à 11 ans profiteront de tarifs réduits. Vous trouverez des épiceries et des restaurants à quelques minutes à pied.
Classé bâtiment respe

In [37]:
weather_df.merge(top_hotels_df).sort_values(['rank','booking_reviews_score'], ascending = [True, False])

,id,city,feels_like.day,pop,delta_with_ideal_temp,pop_%,delta_temp_scaled,pop_scaled,score_to_minimize,rank,hotel_name,booking_url,booking_reviews_score,description,latitude,longitude
14,0,Eguisheim,30.43625,0.12375,0,12.375,-1.288998,-0.568045,-1.857044,1,La Grange de Madeleine,https://www.booking.com/hotel/fr/la-grange-de-...,9.7,La Grange de Madeleine est située à Eguisheim....,48.041783,7.306547
8,0,Eguisheim,30.43625,0.12375,0,12.375,-1.288998,-0.568045,-1.857044,1,Fleur de Vigne,https://www.booking.com/hotel/fr/fleur-de-vign...,9.6,Vous pouvez bénéficier d'une réduction Genius ...,48.046113,7.305163
2,0,Eguisheim,30.43625,0.12375,0,12.375,-1.288998,-0.568045,-1.857044,1,Au coup de coeur,https://www.booking.com/hotel/fr/au-coup-de-co...,9.5,"Situé à Eguisheim, l'établissement Au coup de ...",48.043788,7.304967
9,0,Eguisheim,30.43625,0.12375,0,12.375,-1.288998,-0.568045,-1.857044,1,Elsass Design Hygge,https://www.booking.com/hotel/fr/elsass-design...,9.4,"Situé à Eguisheim, l'Elsass Design Hygge propo...",48.042030,7.304598
5,0,Eguisheim,30.43625,0.12375,0,12.375,-1.288998,-0.568045,-1.857044,1,Les chambres du domaine,https://www.booking.com/hotel/fr/les-chambres-...,9.3,"Situé à Eguisheim, en Alsace, avec le marché d...",48.042963,7.307216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,4,Cassis,30.25500,0.19250,0,19.250,-1.288998,-0.196616,-1.485615,5,Best Western Hotel & SPA Coeur De Cassis,https://www.booking.com/hotel/fr/du-grand-jard...,8.1,Cet établissement est à 3 minutes à pied de la...,43.214573,5.540147
84,4,Cassis,30.25500,0.19250,0,19.250,-1.288998,-0.196616,-1.485615,5,Best Western Plus Hôtel la Rade,https://www.booking.com/hotel/fr/la-rade-cassi...,8.0,Cet établissement est à 4 minutes à pied de la...,43.215230,5.533697
94,4,Cassis,30.25500,0.19250,0,19.250,-1.288998,-0.196616,-1.485615,5,"The Originals Boutique, Hôtel Cassitel, Cassis",https://www.booking.com/hotel/fr/interhotel-ca...,7.7,Cet établissement est à 2 minutes à pied de la...,43.213706,5.538958
95,4,Cassis,30.25500,0.19250,0,19.250,-1.288998,-0.196616,-1.485615,5,Hotel Le Golfe,https://www.booking.com/hotel/fr/le-golfe-cass...,7.7,Cet établissement est à 1 minute à pied de la ...,43.214434,5.535295


In [41]:
top_hotels_df.to_csv('hotels_data.csv')